# Import Library

In [2]:
import numpy as np # pustaka untuk numerik
import pandas as pd # pustaka pemrosesan data
import matplotlib.pyplot as plt  # pustaka untuk plotting 
import seaborn as sns   #pustaka untuk visualisasi data
import glob # pustaka untuk file path
import cv2  # pustaka untuk algoritma visi komputer 
import os     # pustaka untuk interaksi dengan Sistem Operasi
import sklearn   # pustaka untuk pembelajaran mesin 

import warnings
# filter warning
warnings.filterwarnings('ignore')

# Load Dataset

In [ ]:
import os
print(os.listdir("../dataset-buah"))
from subprocess import check_output


# Checking Training Dataset

In [ ]:
np.random.seed(1234)

# nama directory untuk data pelatihan
directory_training="../dataset-buah/Training/"    
# nama kelas dari tiap atribut buah
kelas_training=["Nanas","Alpukat","Pisang","Salak","Kelapa","Kiwi",
         "Lemon","Mangga","Jeruk"]
# semua data pelatihan disimpan di array ini 
training_array=[]

img_size=100
for i in kelas_training:
    path=os.path.join(directory_training,i)
    print("path",path)
    class_num=kelas_training.index(i)
    for img in os.listdir(path):        
        img_array=cv2.imread(os.path.join(path,img))
        img_array=cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        training_array.append([img_array,class_num])

# Checking Validation Dataset

In [ ]:
directory_validation="../dataset-buah/Validation/"
kelas_validation=["Nanas","Alpukat","Pisang","Salak","Kelapa","Kiwi",
         "Lemon","Mangga","Jeruk"]

# semua data validasi disimpan di array ini 
validation_array=[]
img_size=100
for i in kelas_validation:
    path=os.path.join(directory_validation,i)
    print("path",path)
    class_num2=kelas_validation.index(i)
    for img in os.listdir(path):
       
        img_array2=cv2.imread(os.path.join(path,img))
        img_array2=cv2.cvtColor(img_array2, cv2.COLOR_BGR2RGB)
        validation_array.append([img_array2,class_num2])

# Checking All Sub-folder Dataset

In [ ]:
# ekstraksi fitur dari array pelatihan
fruits_array_train = [features for features, label in training_array]

# definisikan lokasi dan nama buah
location = [[1, 500, 1150], [1500, 2000, 2500], [3000, 3500, 4000]]
nama_buah = ["Nanas", "Alpukat", "Pisang", "Salak", "Kelapa", "Kiwi", "Lemon", "Mangga", "Jeruk"]

# Iterasi lokasi dan nama buah
for loc, name in zip(location, fruit_names):
    # Buat subplot untuk setiap lokasi
    plt.subplots(figsize=(8, 8))
    for i, index in enumerate(loc):
        # tambahkan subplot untuk setiap buah
        plt.subplot(1, 3, i + 1)
        plt.imshow(fruits_array_train[index])
        plt.title(name)
        plt.axis("off")

# Preprocessing Data

In [ ]:
import random
random.shuffle(training_array)

# Pengacakan array data pelatihan dan validasi
random.shuffle(training_array)
random.shuffle(validation_array)

# ekstraksi fitur dan label dari array yang teracak
X_train, Y_train = zip(*training_array)
X_test, Y_test = zip(*validation_array)

# konversi list ke bentuk array Numpy
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

# Reshape dan normalisasi array
X_train = X_train.reshape(-1, img_size, img_size, 3)
X_train = X_train / 255
X_test = X_test.reshape(-1, img_size, img_size, 3)
X_test = X_test / 255

print("shape of X_train=", X_train.shape)
print("shape of X_test=", X_test.shape)



In [ ]:
from keras.utils import to_categorical
# konversi array ke bentuk kelas label sebagai matrix binary
Y_train=to_categorical(Y_train,num_classes=9)
Y_test=to_categorical(Y_test,num_classes=9)

In [ ]:
Y_train.shape
X_train.shape

# Split Dataset to Training and Validation

In [ ]:
# Bagi dataset untuk pelatihan dan validasi 
from sklearn.model_selection import  train_test_split
x_train,x_val,y_train,y_val=train_test_split(X_train,Y_train,test_size=0.2,random_state=42)

# Create Model

In [ ]:
import itertools

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import (
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPool2D,
)
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# buat model neural network Convolutional 

model = Sequential()
model.add(Conv2D(16, (3, 3), padding="Same", activation="relu", input_shape=(100, 100, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(32, (3, 3), padding="Same", activation="relu"))
model.add(MaxPool2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(64, (3, 3), padding="Same", activation="relu"))
model.add(MaxPool2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.35))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(9, activation="softmax"))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
#Kompilasi model yang sudah dibangun 
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

epochs = 10
batch_size = 18
model.summary()  #tampilkan hasilnya

# Training Model

In [ ]:
#  Data augmentation dapat membantu meningkatkan kemampuan generalisasi model dengan membuatnya lebih tahan terhadap variasi dalam data.
datagen=ImageDataGenerator(featurewise_center=False, #hitung rerata nilai piksel di seluruh dataset latih,tiap citra kemudian dikurangi rerata ini.
                           samplewise_center=False,  #hitung rerata nilai piksel hanya pada sampel tersebut, tiap citra dikurangi rerata ini
                           featurewise_std_normalization=False, #membagi input data dengan seluruh standard deviasi
                           samplewise_std_normalization=False,  #membagi input data dengan standard deviasi pada sampel tsb
                          zca_whitening=False,  # mengurangi redundansi dari data yang terkorelasi
                           rotation_range=0.5,    #putar 5 derajat
                            zoom_range=0.5,        #zoom in-out 5%
                           width_shift_range=0.5, #geser 5% secara horizontal
                           height_shift_range=0.5, #geser 5% vertikal 
                           horizontal_flip=False,  # membalik citra secara horizontal
                           vertical_flip=False, # membalik citra secara vertikal
                           )
datagen.fit(x_train)

# menerapkan model yang sudah dibuat ke dataset pelatihan dan validasi
history=model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),epochs=epochs,
                            validation_data=(x_val,y_val),steps_per_epoch=x_train.shape[0]//batch_size
                           )

# Save Model

In [ ]:
from keras.models import load_model
# simpan model Neural Network 
model.save("model_buah")

# Plotting Data

In [ ]:
# plot akurasi terhadap epoch 
plt.plot(history.history["val_accuracy"],color="r",label="val_acc")
plt.title("Accuracy Graph")
plt.xlabel("number of epochs")
plt.ylabel("accuracy")
plt.legend()
plt.grid()
plt.show()


In [ ]:
#Plot loss terhadap epoch

print(history.history.keys())

plt.style.use('fivethirtyeight')

xepochs = [i+1 for i in range(0, len(history.history['loss']))]
plt.figure(figsize=(5,3))

# Loss
#plt.ylim([-0.1,0.5])
plt.plot(xepochs, history.history['loss'])
plt.plot(xepochs, history.history['val_loss'])
plt.xticks(xepochs)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

# akurasi 

plt.figure(figsize=(5,3))
plt.plot(xepochs, history.history['accuracy'])
plt.plot(xepochs, history.history['val_accuracy'])
plt.xticks(xepochs)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

In [ ]:
# gambarkan matriks konfusi untuk data pelatihan
from sklearn.metrics import confusion_matrix

#matriks konfusi
y_pred=model.predict(x_val)
y_pred_classes=np.argmax(y_pred,axis=1)
y_true=np.argmax(y_val,axis=1)

# hitung matriks konfusi
conf_mat=confusion_matrix(y_true,y_pred_classes)

# plot matriks konfusi
fruit_names=["Nanas","Alpukat","Pisang","Salak","Kelapa","Kiwi",
         "Lemon","Mangga","Jeruk"]

f,ax=plt.subplots(figsize=(10,9))
sns.heatmap(conf_mat,annot=True,fmt='.0f')
ax.set_xticklabels(fruit_names)
ax.set_yticklabels(fruit_names)
plt.show()

In [ ]:
## gambarkan matriks konfusi untuk data validasi
y_pred2=model.predict(X_test)
y_pred_classes2=np.argmax(y_pred2,axis=1)
y_true2=np.argmax(Y_test,axis=1)

# hitung matriks konfusi
conf_mat2=confusion_matrix(y_true2,y_pred_classes2)

# plot matriks konfusi
f,ax=plt.subplots(figsize=(10,9))
sns.heatmap(conf_mat2,annot=True,fmt=".0f")
ax.set_xticklabels(fruit_names)
ax.set_yticklabels(fruit_names)
plt.show()

# Testing and Prediction

In [ ]:
import tensorflow.keras as k
PREDICTION_PATH="C:/Users/ASUS ROG/Documents/klasifikasi_buah/input/testing-prediksi"
# coba prediksi citra

images_for_prediction = [filename for filename in sorted(os.listdir(PREDICTION_PATH)) if filename.endswith(".jpg")]


for filename in images_for_prediction:
    loaded_image = k.preprocessing.image.load_img(path=PREDICTION_PATH+'/'+filename, target_size=(img_size,img_size,3))
    #konversi array dan resample dibagi 255
    img_array = k.preprocessing.image.img_to_array(loaded_image) / 255.

    # tambahkan dimensi 
    img_np_array = np.expand_dims(img_array, axis = 0)
    
    predictions = model.predict(img_np_array)
    classidx = np.argmax(predictions[0])
    label = nama_buah[classidx]

    predictions_pct = ["{:.2f}%".format(prob * 100) for prob in predictions[0] ]
    print(dict(zip(nama_buah, predictions_pct)) )
    print("Prediction: %s (class %s) %s" % (label, classidx, predictions_pct[classidx])) 

    plt.figure(figsize=(3,4))
    plt.imshow(img_array)
    plt.title("%s %s" % (label, predictions_pct[classidx]))
    plt.show()

plt.gcf().clear()

